In [21]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import secret
import json

baseurl= "https://geocoder.api.here.com/6.2/geocode.json?"
basepath= "./1994-1996/"
fileName = "1994-1996"
baseutil = "./Util/"
encoding='UTF-8'

In [24]:
file = open(basepath+fileName+".csv", "r")

base = pd.read_csv(file, sep=";", names=['Sigla da Matéria Legislativa', 'Ano da propositura', 'Número da propositura', 'Ano da propositura', 'Ementa (texto em linha única)','Ind. Destinatário', 'Ind. Motivo', 'Ind. Endereco' , 'Ind. Adm.Regional', 'Ind. Bairro' , 'Nome', 'Partido (último)'], encoding=encoding)

c:\users\renan\documents\lab\indicacoes\env\lib\site-packages\pandas\io\parsers.py:678: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 453: character maps to <undefined>

In [ ]:
#baseNotNan = base[base["Ind. Endereco"].notnull()]
#baseNotNan.to_csv(basepath+"91-93withNotNan.csv", sep=';', encoding='utf-8')


In [ ]:
#baseWithNan = base[base["Ind. Endereco"].isnull()]
#baseWithNan.to_csv(basepath+"91-93withNan.csv", sep=';', encoding='utf-8')

In [ ]:
#baseWithNanAndNei = baseWithNan[baseWithNan["Ind. Bairro"].notnull()]
#baseWithNanAndNei.to_csv(basepath+"91-93withNanNei.csv", sep=';', encoding='utf-8')

In [ ]:
#baseWithTotalNan = baseWithNan[baseWithNan["Ind. Bairro"].isnull()]
#baseWithTotalNan.to_csv(basepath+"91-93withTotalNan.csv", sep=';', encoding='utf-8')

In [ ]:
#TotalCount = len(base.index)
#WithAddress = len(baseNotNan.index)
#WithNoAddress = len(baseWithNan.index)
#WithNoAddressNeighbors = len(baseWithNanAndNei.index)
#WithNOAddressNoNeighbors = len(baseWithTotalNan.index)

#print ("Total: %s \n Quantidade com Endereco: %s \n Quantidade sem Endereco: %s \n Quantidade sem endereco com Bairro: %s \n Quantidade sem endereco e sem bairro: %s" % (TotalCount,WithAddress, WithNoAddress, WithNoAddressNeighbors, WithNOAddressNoNeighbors))
#print ("\nTotal em Porcentagem: %s \n Total com Endereco: %s \n Quantidade sem Endereco: %s \n Quantidade sem endereco com Bairro: %s \n Quantidade sem endereco e sem bairro: %s" % (TotalCount, (WithAddress/TotalCount)*100, (WithNoAddress/TotalCount)*100, (WithNoAddressNeighbors/TotalCount)*100, (WithNOAddressNoNeighbors/TotalCount)*100))


In [ ]:

bairrosFile = open(baseutil+"Bairros.csv", "r")
bairros = pd.read_csv(bairrosFile, sep=",", names=['Bairros'], encoding=encoding)

abrvFile = open(baseutil+"Abrev.csv", "r")
abrev = pd.read_csv(abrvFile, sep=";", names=['Abreviacoes', 'Referencia'], encoding=encoding)

enderecoLoc = ["Ind. Endereco", "Ind. Bairro"]
address = base.loc[:, enderecoLoc]


In [ ]:
#abrvs = dict()

#for index, row in address.iterrows():
#    if type(row[0]) is str:
#        abrv = row[0].split(",")
        
#        for ab in abrv[2:]:
#            if ab in abrvs.keys():
#                abrvs[ab] += 1
#                continue
#            abrvs[ab] = 1

#abrvs = sorted(abrvs.items(), key=lambda value: value[1], reverse=True)
#ad = pd.Series(pd.toDataFrmae(abrvs)).to_frame()
#ad = pd.DataFrame(abrvs)
#ad.to_csv(basepath+"Abreviacoes.csv", sep=';', encoding='utf-8')
#print ("Done!")
    

In [ ]:


def isAbrev(name):
    try:
        for index, row in abrev.iterrows():
            if type(row[0]) is str:
                if name.lower().strip() == row[0].lower().strip():
                    return row[1] + " "
    except:
        pass
    return None

def isNeighbor(name):
    try:
        for index, row in bairros.iterrows():
            if type(row[0]) is str:
                if name.lower().strip() == row[0].lower().strip():
                    return row[1]
    except:
        pass
    return None
i = 0

localization = { 'Latitude': list(), "Longitude": list()}
for index, row in address.iterrows():
    latitude, longitude = "", ""
    if i == 10:
        break
    if type(row[0]) is str:
        adressByPart = row[0].split(",")
        street = ""
        neigh = ""
        abr = ""
        second = ""
        for parts in adressByPart:        
            if isAbrev(parts) is not None:
                abr = isAbrev(parts) + abr 
            elif isNeighbor(parts) is not None:
                neigh = isNeighbor(parts)
            else:
                street = street + parts
        if len(row) > 1:
            if str(row[1]) != "nan":
                neigh = row[1]
        adr = abr + street + neigh
        
        latitude, longitude = getDirections(adr)
        
    i+=1
    print(i)
    localization['Latitude'].append(latitude)
    localization['Longitude'].append(longitude)
        

lladr = pd.DataFrame(localization)
result = pd.concat([base, lladr], axis = 1)
result.to_csv(basepath+fileName+"-Localizado.csv", sep=';', encoding=encoding)
print("Done!")


In [ ]:
def getDirections(adr):
    adr = adr+"+Sao+Paulo+SP"
    adr = adr.replace(' ','+')
    request = baseurl+"app_id="+secret.app_id+"&app_code="+secret.app_code+"&searchtext="+adr
    response = (requests.get(request)).json()
    latitude = ""
    longitude = ""

    if len(response["Response"]["View"]) > 0:
        latitude  =  response["Response"]["View"][0]["Result"][0]["Location"]["NavigationPosition"][0]["Latitude"]
        longitude = response["Response"]["View"][0]["Result"][0]["Location"]["NavigationPosition"][0]["Longitude"]
    return str(latitude), str(longitude)


In [4]:
file.close()